# Modules
**___**
## Preprocessing.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

df = pd.read_csv("train.csv")
df_test = pd.read_csv('test.csv')
df_y = pd.read_csv("train_label.csv")
# print(df.loc[0])

df_train = df.drop(columns=['is_canceled', 'adr', 'reservation_status', 'reservation_status_date'])
df_hidden = df[['is_canceled', 'adr', 'reservation_status', 'reservation_status_date']]

df_cat = pd.concat([df_train, df_test], ignore_index = True)
# print(df_cat.loc[112600])

In [2]:
df_req = df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month',
             'is_canceled', 'adr', 'stays_in_weekend_nights', 'stays_in_week_nights']]
df_req = df_req.assign(rev = lambda x: (x.is_canceled==0)*x.adr*(x.stays_in_weekend_nights+x.stays_in_week_nights))

# format date
sort_dict_m = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 'May':'05', 'June':'06',
               'July':'07', 'August':'08', 'September':'09', 'October':'10', 'November':'11', 'December':'12'}
sort_dict_d = {1:'01', 2:'02', 3:'03', 4:'04', 5:'05', 6:'06', 7:'07', 8:'08', 9:'09'}
df_req = df_req.replace({'arrival_date_month':sort_dict_m})
df_req = df_req.replace({'arrival_date_day_of_month':sort_dict_d})
df_req = df_req.assign(arrival_date = lambda x: x["arrival_date_year"].astype(str)+'-'+x["arrival_date_month"]+'-'+x["arrival_date_day_of_month"].astype(str))

rev = df_req[['arrival_date', 'rev']]
rev = rev.groupby(['arrival_date']).sum()
print(rev)

                       rev
arrival_date              
2015-07-01    20311.186621
2015-07-02    16530.645277
2015-07-03    12966.714164
2015-07-04    17480.654256
2015-07-05    19591.458478
...                    ...
2017-03-27    26217.381380
2017-03-28    16185.177703
2017-03-29    24002.255525
2017-03-30    33095.297394
2017-03-31    36062.103164

[640 rows x 1 columns]


In [3]:
# format date
df_cat = df_cat.replace({'arrival_date_month':sort_dict_m})
df_cat = df_cat.replace({'arrival_date_day_of_month':sort_dict_d})
df_cat = df_cat.assign(arrival_date = lambda x: x["arrival_date_year"].astype(str)+'-'+x["arrival_date_month"]+'-'+x["arrival_date_day_of_month"].astype(str))
# print(df_cat.loc[112600])

In [4]:
df_encoded = pd.get_dummies(df_cat, columns=["hotel", "arrival_date_month", "meal", "country", "market_segment",
                                             "distribution_channel", "reserved_room_type", "assigned_room_type",
                                             "deposit_type", "customer_type", "agent", "company"], dummy_na = True)
# append 0 to missing values
idx = df_encoded[df_encoded['children'].isnull()].index.tolist()
df_encoded["children"][idx] = 0

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
df_train = df_encoded.iloc[:91531, :].reset_index(drop=True)
df_test = df_encoded.iloc[91531:, :].reset_index(drop=True)

iscadr = df[['is_canceled', 'adr']]
df_x = df_train.drop(columns=['ID', 'arrival_date', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month'])

# NAMES
# --------------------------------------------------
# df       : original training data
# df_train : dummy-coded training data with only columns that the test data also contains
# df_x     : df_train without columns containing separate date information (except for 'arrival_date_month')
#            (Train the models with this!)
# df_y     : labels of training data (rank column)
# df_test  : testing data
# df_x_test: testing data without irrevelant variables
# iscadr   : is_canceled and adr columns (training data)
# rev      : revenue column (calculated with training data)

___
## 5-fold Validation

In [23]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

def rev_calc(isc_pred, adr_pred, df):
    """Calculate revenue from predicted is_canceled (bool) and adr"""
    rev_pred = np.zeros(np.shape(isc_pred)[0])
    for i in range (np.shape(isc_pred)[0]):
        rev_pred[i] = (isc_pred[i]==0)*adr_pred[i]*(df['stays_in_weekend_nights'][i] + df['stays_in_week_nights'][i])
    return rev_pred

kf = KFold(n_splits=5, shuffle=True)
for train_index, val_index in kf.split(df_x, iscadr):
    X_train, X_val = df_x.iloc[train_index, :].reset_index(drop=True), df_x.iloc[val_index, :].reset_index(drop=True)
    y_train, y_val = iscadr.iloc[train_index, :].reset_index(drop=True), iscadr.iloc[val_index, :].reset_index(drop=True)
    
    # LogReg on **is_canceled**
    lgc_isc = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train['is_canceled'])
    isc_pred = lgc_isc.predict(X_val)
    print("E_in_isc =", lgc_isc.score(X_train, y_train['is_canceled']))
    
    # DecisionTree on **adr**
    dt_adr = DecisionTreeRegressor(random_state=0).fit(X_train, y_train['adr'])
    dt_adr_pred = dt_adr.predict(X_val)
    print("E_in_adr =", dt_adr.score(X_train, y_train['adr']))
    
    # Calculate **revenue**
    rev_pred = rev_calc(isc_pred, adr_pred)
    df_rev_pred = pd.DataFrame({'rev_pred': rev_pred})
    df_date = df_train['arrival_date'][val_index].reset_index(drop=True)
    ## Sum daily revenues
    X_val_rev = pd.concat([df_date, df_rev_pred], join='outer', axis=1)
    X_val_rev = X_val_rev.groupby(['arrival_date']).sum()
    print(X_val_rev)
    
    # Calculate **revenue** for validation set
    isc_true = np.array(y_val['is_canceled'])
    adr_true = np.array(y_val['adr'])
    rev_true = rev_calc(isc_true, adr_true)
    df_rev_true = pd.DataFrame({'rev_true': rev_true})
    ## Sum daily revenues
    y_val_rev = pd.concat([df_date, df_rev_true], join='outer', axis=1)
    y_val_rev = y_val_rev.groupby(['arrival_date']).sum()
    print(y_val_rev)
    
    print("MAE =", mean_absolute_error(y_val_rev, X_val_rev))
    print('___')

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


E_in_isc = 0.8333333333333334
E_in_adr = 0.9801277145712805
[   0.         0.         0.      ...  167.90625  120.125   1634.9375 ]
                  rev_pred
arrival_date              
2015-07-01   -5.804302e+16
2015-07-02    4.262438e+03
2015-07-03    1.267750e+03
2015-07-04    2.240719e+03
2015-07-05    2.040188e+03
...                    ...
2017-03-27    7.060188e+03
2017-03-28    8.658656e+03
2017-03-29    8.649281e+03
2017-03-30    3.747578e+03
2017-03-31    1.261797e+04

[640 rows x 1 columns]
                 rev_true
arrival_date             
2015-07-01    8216.673516
2015-07-02    3092.748770
2015-07-03    1571.172255
2015-07-04    1177.130326
2015-07-05    1569.712671
...                   ...
2017-03-27    8321.661129
2017-03-28    7100.715324
2017-03-29    6293.767895
2017-03-30    3978.276169
2017-03-31    7171.901411

[640 rows x 1 columns]
MAE = 1167649435874885.0
___


C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


KeyboardInterrupt: 

___
## Postprocessing.
### 1. Calculate **revenue**.

In [ ]:
def rev_calc(isc_pred, adr_pred):
    """Calculate revenue from predicted is_canceled (bool) and adr"""
    rev_pred = np.zeros(np.shape(isc_pred)[0])
    for i in range (np.shape(isc_pred)[0]):
        rev_pred[i] = (isc_pred[i]==0)*adr_pred[i]*(df_test['stays_in_weekend_nights'][i] + df_test['stays_in_week_nights'][i])

rev_pred = rev_calc(isc_pred, adr_pred)

In [ ]:
# Sum daily revenues
df_test.insert(np.shape(df_test)[1], 'rev', rev_pred)    # append column
df_test = df_test.replace({'arrival_date_month':sort_dict_m})
df_test = df_test.replace({'arrival_date_day_of_month':sort_dict_d})
df_test = df_test.assign(arrival_date = lambda x: x["arrival_date_year"].astype(str)+'-'+x["arrival_date_month"]+'-'+x["arrival_date_day_of_month"].astype(str))

df_test_rev = df_test[['arrival_date', 'rev']]
df_test_rev = df_test_rev.groupby(['arrival_date']).sum()
print(df_test_rev)

### 2. From **revenue** to **ranking**.

In [ ]:
rank = df_y['label']

poly = PolynomialFeatures(2)    # default degree: 2, may change
rev_inter = poly.fit_transform(rev)
reg = LinearRegression(normalize=True).fit(rev_inter, rank)
print(reg.score(rev_inter, rank))

df_test_rev_inter = poly.fit_transform(df_test_rev)
rank_pred = reg.predict(df_test_rev_inter)
for i in range(np.shape(rank_pred)[0]):
    if (rank_pred[i] < 0):
        rank_pred[i] = 0
    elif (rank_pred[i] > 9):
        rank_pred[i] = 9
    else:
        rank_pred[i] = round(rank_pred[i])

In [ ]:
# Generate submission file
ans = df_test_rev
ans['label'] = rank_pred
ans.drop(columns=['rev'], inplace=True)
ans.to_csv('out.csv')